In [ ]:
# Install transformers, sentencepiece, and datasets libraries
!pip install transformers
!pip install sentencepiece
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
!pip install evaluate
!pip install jiwer
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.2 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, T5ForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, Dataset
import evaluate
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from jiwer import wer

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
from jiwer import wer
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
class BPETokenizer:
    def __init__(self, vocab_size=100000000, special_tokens=None):
        if special_tokens is None:
            special_tokens = ['<pad>', '<unk>', '<s>', '</s>']
        self.vocab_size = vocab_size
        self.special_tokens = special_tokens
        self.vocab = {}
        self.merges = {}
        self.token_to_id = {}
        self.id_to_token = {}
        for token in special_tokens:
            self._add_token(token)

    def _add_token(self, token):
        if token not in self.token_to_id:
            idx = len(self.token_to_id)
            self.token_to_id[token] = idx
            self.id_to_token[idx] = token

    def train(self, corpus):
        # Step 1: Add all unique characters to the vocab
        chars = set()
        for text in corpus:
            chars.update(text.replace(' ', ''))  # Remove spaces while collecting chars
        for char in chars:
            self._add_token(char)

        print(f"Initial vocabulary size: {len(self.token_to_id)}")

        # Step 2: Remove spaces and tokenize into lists of characters
        corpus = [text.replace(' ', '') for text in corpus]
        tokenized_corpus = [list(text) for text in corpus]

        while len(self.token_to_id) < self.vocab_size:
            # Step 3: Count all adjacent pairs
            pair_counts = {}
            for tokens in tokenized_corpus:
                for i in range(len(tokens) - 1):
                    pair = (tokens[i], tokens[i + 1])
                    pair_counts[pair] = pair_counts.get(pair, 0) + 1

            if not pair_counts:
                break

            # Step 4: Get the most frequent pair
            sorted_pairs = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)
            print("Most frequent pairs and their counts:", sorted_pairs[:5])
            best_pair = sorted_pairs[0][0]
            new_token = ''.join(best_pair)
            print(f"Best pair found: {best_pair}, New token: {new_token}")

            # Step 5: Add the new token to vocab and record the merge
            self._add_token(new_token)
            self.merges[best_pair] = new_token

            # Step 6: Replace best_pair in corpus with new_token
            new_tokenized_corpus = []
            for tokens in tokenized_corpus:
                new_tokens = []
                i = 0
                while i < len(tokens):
                    if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == best_pair:
                        new_tokens.append(new_token)
                        i += 2
                    else:
                        new_tokens.append(tokens[i])
                        i += 1
                new_tokenized_corpus.append(new_tokens)
            tokenized_corpus = new_tokenized_corpus

        print(f"Final vocabulary size: {len(self.token_to_id)}")
        print(f"Final token to ID mapping (sample): {list(self.token_to_id.items())[:10]}")

    def encode(self, text):
        text = text.replace(' ', '')  # Ensure consistency with training
        tokens = list(text)
        changed = True
        while changed:
            changed = False
            i = 0
            while i < len(tokens)-1:
                pair = (tokens[i], tokens[i+1])
                if pair in self.merges:
                    tokens[i:i+2] = [self.merges[pair]]
                    changed = True
                else:
                    i += 1
        ids = [self.token_to_id.get(token, self.token_to_id['<unk>']) for token in tokens]
        return ids

    def decode(self, ids):
        return ''.join([self.id_to_token.get(i, '<unk>') for i in ids])


In [ ]:
# Positional Encoding
class PositionalEncoding:
    def __init__(self, d_model, max_len=512):
        self.d_model = d_model
        self.max_len = max_len
        self.pe = torch.zeros(max_len, d_model)

        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def __call__(self, x):
        # x shape: [seq_len, d_model]
        seq_len = x.size(0)
        if seq_len > self.max_len:
            raise ValueError(f"Sequence length {seq_len} exceeds maximum length {self.max_len}")
        return x + self.pe[:seq_len, :]


In [ ]:
# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

class VerilogToCCodeDataset(Dataset):
    def __init__(self, verilog_file, c_code_file, tokenizer, max_length=512):
        with open(verilog_file, 'r') as vf:
            self.verilog_lines = [line.strip() for line in vf.readlines()]
        with open(c_code_file, 'r') as cf:
            self.c_code_lines = [line.strip() for line in cf.readlines()]

        assert len(self.verilog_lines) == len(self.c_code_lines), "Mismatch in number of Verilog and C code lines"
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.verilog_lines)

    def __getitem__(self, idx):
        verilog_code = self.verilog_lines[idx]
        c_code = self.c_code_lines[idx]

        # Tokenize input and target sequences
        input_encoding = tokenizer(verilog_code, padding='max_length', truncation=True, max_length=self.max_length)
        target_encoding = tokenizer(c_code, padding='max_length', truncation=True, max_length=self.max_length)

        # Adding positional encodings manually
        input_encoding['position_ids'] = torch.arange(self.max_length)
        target_encoding['position_ids'] = torch.arange(self.max_length)

         # Merge token embeddings with position embeddings
        with torch.no_grad():
            input_embeddings = self.model.encoder.embed_tokens(input_ids)  # Token embeddings
            position_embeddings = self.model.encoder.embed_positions(position_ids)  # Positional embeddings
            input_embeddings += position_embeddings  # Combine embeddings

        return {
            'input_ids': torch.tensor(input_encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(input_encoding['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(target_encoding['input_ids'], dtype=torch.long),
            'decoder_attention_mask': torch.tensor(target_encoding['attention_mask']),
            'position_ids': input_encoding['position_ids']
        }

# Dataset file paths
train_src = '/content/drive/MyDrive/Code_transfer/train.src'
train_tgt = '/content/drive/MyDrive/Code_transfer/train.tgt'
valid_src = '/content/drive/MyDrive/Code_transfer/valid.src'
valid_tgt = '/content/drive/MyDrive/Code_transfer/valid.tgt'
test_src = '/content/drive/MyDrive/Code_transfer/test.src'
test_tgt = '/content/drive/MyDrive/Code_transfer/test.tgt'

# Load datasets
train_dataset = VerilogToCCodeDataset(train_src, train_tgt, tokenizer)
valid_dataset = VerilogToCCodeDataset(valid_src, valid_tgt, tokenizer)
test_dataset = VerilogToCCodeDataset(test_src, test_tgt, tokenizer)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
import csv

def save_vocab_encodings_to_csv(bpe_tokenizer, model, file_path="vocab_encodings.csv", max_tokens=10000, d_model=512):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare embedding lookup
    token_to_id = bpe_tokenizer.token_to_id
    id_to_token = bpe_tokenizer.id_to_token

    vocab_size = len(token_to_id)
    print(f"Vocab size: {vocab_size}")

    # Create tensor of all ids
    token_ids = torch.tensor(list(range(vocab_size)), dtype=torch.long).to(device)

    # Get BPE embeddings (from model's shared embedding layer)
    bpe_embeddings = model.shared(token_ids).detach().cpu()  # shape: (vocab_size, d_model)

    # Generate positional encodings
    positional_encoder = PositionalEncoding(d_model=d_model, max_len=vocab_size)
    pos_encodings = positional_encoder.pe[:vocab_size]  # shape: (vocab_size, d_model)

    # Prepare data for saving
    output_data = []
    output_data.append(['Token', 'ID', 'BPE Embedding [0:5]', 'Pos Enc [0:5]'])  # Header

    for idx in range(vocab_size):
        token = id_to_token[idx]
        bpe_vec = [f'{v:.4f}' for v in bpe_embeddings[idx][:5].tolist()]
        pos_vec = [f'{v:.4f}' for v in pos_encodings[idx][:5].tolist()]
        output_data.append([token, idx, str(bpe_vec), str(pos_vec)])

    # Write to CSV file
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(output_data)
    print(f"Data saved to {file_path}")

# 🔁 Call the function (after tokenizer training and model initialization)
save_vocab_encodings_to_csv(train_dataset.bpe_tokenizer, model)


In [ ]:
def concatenate_bpe_and_positional_encodings(bpe_tokenizer, model, d_model=512, max_tokens=10000):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare embedding lookup
    token_to_id = bpe_tokenizer.token_to_id
    id_to_token = bpe_tokenizer.id_to_token

    vocab_size = len(token_to_id)
    print(f"Vocab size: {vocab_size}")

    # Create tensor of all ids
    token_ids = torch.tensor(list(range(vocab_size)), dtype=torch.long).to(device)

    # Get BPE embeddings (from model's shared embedding layer)
    bpe_embeddings = model.shared(token_ids).detach().cpu()  # shape: (vocab_size, d_model)

    # Generate positional encodings
    positional_encoder = PositionalEncoding(d_model=d_model, max_len=vocab_size)
    pos_encodings = positional_encoder.pe[:vocab_size]  # shape: (vocab_size, d_model)

    # Concatenate BPE embeddings with positional encodings
    concatenated_embeddings = torch.cat((bpe_embeddings, pos_encodings), dim=-1)  # shape: (vocab_size, d_model * 2)

    return concatenated_embeddings, id_to_token


In [ ]:
import csv

def save_concatenated_vocab_encodings_to_csv(bpe_tokenizer, model, file_path="concatenated_vocab_encodings.csv", d_model=512, max_tokens=10000):
    concatenated_embeddings, id_to_token = concatenate_bpe_and_positional_encodings(bpe_tokenizer, model, d_model=d_model, max_tokens=max_tokens)

    # Prepare data for saving
    output_data = []
    output_data.append(['Token', 'ID', 'Concatenated Embedding [0:5]'])  # Header

    for idx in range(len(id_to_token)):
        token = id_to_token[idx]
        concatenated_vec = [f'{v:.4f}' for v in concatenated_embeddings[idx][:5].tolist()]
        output_data.append([token, idx, str(concatenated_vec)])

    # Write to CSV file
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(output_data)
    print(f"Data saved to {file_path}")

# Call the function (after tokenizer training and model initialization)
save_concatenated_vocab_encodings_to_csv(train_dataset.bpe_tokenizer, model)



In [ ]:

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=100,
    logging_first_step=True,
    # Add verbose logging
    logging_strategy="steps",  # Log at every step
    report_to=["none"],  # Disable wandb logging
)

# Trainer for fine-tuning
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.569300,0.262519
2,0.260300,0.215582
3,0.194800,0.194325
4,0.190300,0.181157
5,0.171400,0.170494
6,0.155500,0.163121
7,0.149900,0.157839
8,0.137600,0.151884
9,0.129100,0.148633
10,0.140100,0.145661


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json')

In [ ]:
# Testing phase
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
model.eval()
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Evaluation
predictions = []
references = []
smooth_func = SmoothingFunction().method1
total_bleu = 0
total_wer = 0

for idx, sample in enumerate(test_loader):
    if idx >= 50:  # Evaluate only the first 50 samples
        break
    input_ids = sample['input_ids'].to(model.device)
    labels = sample['labels'].to(model.device)
    attention_mask = sample['attention_mask'].to(model.device)

    # Generate output
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=512)
    prediction = tokenizer.decode(output[0], skip_special_tokens=True)
    reference = tokenizer.decode(labels[0], skip_special_tokens=True)

    predictions.append(prediction)
    references.append(reference)

    # Compute WER and BLEU
    wer_score = wer(reference, prediction)
    bleu_score = sentence_bleu([reference.split()], prediction.split(), smoothing_function=smooth_func)

    total_wer += wer_score
    total_bleu += bleu_score

    print(f"Sample {idx+1}:")
    print(f"Source Code (Verilog): {sample['input_ids']}")
    print(f"Reference Code (C): {reference}")
    print(f"Predicted Code (C): {prediction}")
    print(f"WER: {wer_score:.4f}")
    print(f"BLEU Score: {bleu_score:.4f}")
    print("="*50)

# Average scores
average_wer = total_wer / 50
average_bleu = total_bleu / 50

print(f"\nAverage WER for 50 samples: {average_wer:.4f}")
print(f"Average BLEU for 50 samples: {average_bleu:.4f}")

Sample 1:
Source Code (Verilog): tensor([[    1,  2978,  1241,    67,  1153,   261,   269,   513,    67,   830,
            79,    16,   269,   513,    67, 16114,    16,   269,   513,    67,
          1937,    16,   269,   513,    67,  8734,    16,   269,   513,    67,
         20390,    16,   269,   513,    67,  1672,    16,   269,  1026, 10196,
            16,   269, 27331, 14055,  2686,    16,   269,  1022,   337, 30752,
            21,    16,   269,  1022,   337, 30752,    21,    67,   438,    67,
            90,  1236,    16,   269,   513,    67,  2463,   269, 11272,   269,
          1569,   513,    67,   882,    67,  2556,    81,    67,  2019,    21,
           273,  2593, 14271,    21,    31,   269,  1569,   513,    67,   882,
            67,  2556,    81,    67,  2019,    22,   273,  2593, 14271,    22,
            31,   269,  1569,   513,    67,   882,    67,  2556,    81,    67,
          2019,    23,   273,  2593, 14271,    24,    31,   269,  1569,   513,
            67,   8

In [ ]:
from jiwer import wer

def calculate_wer(predictions, references):
    wer_scores = []
    for pred, ref in zip(predictions, references):
        score = wer(ref, pred)  # WER for each pair
        wer_scores.append(score)
    return sum(wer_scores) / len(wer_scores), wer_scores  # Average WER and individual scores

# Calculate WER
average_wer, wer_scores = calculate_wer(predictions, references)
print(f"Total WER: {average_wer}")


Total WER: 0.39018546409029625


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(predictions, references):
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        ref_tokens = ref.split()
        pred_tokens = pred.split()
        bleu = sentence_bleu([ref_tokens], pred_tokens)  # BLEU for each pair
        bleu_scores.append(bleu)
    return sum(bleu_scores) / len(bleu_scores), bleu_scores  # Average BLEU score and individual scores

# Calculate BLEU score
average_bleu, bleu_scores = calculate_bleu(predictions, references)
print(f"Total BLEU Score: {average_bleu}")


Total BLEU Score: 0.6730903109079134
